In [18]:
import numpy as np
import time

In [31]:
def KMeans(K, data):
    startTime = time.time()
    
    d = np.size(data, axis=0)
    N = np.size(data, axis=1)
    I = np.eye(K)
    
    #Centra grup (losowe wektory z danych)
    #R = np.take(data, np.random.choice(np.arange(0,N), K, replace=False), axis=1)
    R = np.empty([d,K])
    choices = np.random.choice(np.arange(0,N), K, replace=False)
    for i in range(K):
        R[:,i] = data[:, choices[i]]
    #Przynależnosc do grupy (na poczatku wszyscy do zero)
    C = np.zeros((1,N), dtype=np.int64)
    #macierzPrzynaleznosci = np.zeros((N,K))

    iteration = 0
    groupsChanged = True
    while groupsChanged:
        iteration += 1
        #Interesuje nas minimalna wartość <r,r> -2<u,r>, gdzie u to wektor z danych a r to jakies centrum
        iloczynySkalarne = -2. * np.dot(data.T, R)
        R**=2
        kwadratyDlugosciR = np.sum(R, axis=0, keepdims=True)
        iloczynySkalarne += kwadratyDlugosciR

        #Dla kazdego wektora z danych wybieramy najblizszy wektor z R i aktualizujemy grupy
        newC = np.argmin(iloczynySkalarne, axis=1)
        groupsChanged = not np.array_equal(C, newC)
        C = newC

        #Obliczamy srodki ciezkosci dla kazdej grupy
        macierzPrzynaleznosci = np.take(I, C, axis=0)
        #Sumy danych w każdej z grup
        R = np.dot(data, macierzPrzynaleznosci)
        liczebnosciGrup = np.maximum(np.sum(macierzPrzynaleznosci, axis=0, keepdims=True), 1)
        R /= liczebnosciGrup

    endTime = time.time()
    print("Iterations:", iteration)
    print("K-Means done in %s seconds" % (endTime - startTime))
    return (C, R)

In [20]:
#Wczytujemy wszystkie transakcje do listy
transactions = [[int(id) for id in line.split()] for line in open("kosarak.dat")]

In [21]:
occurences = {}

for i in range(len(transactions)):
    for j in range(len(transactions[i])):
        if not transactions[i][j] in occurences:
            occurences[transactions[i][j]] = 0
        occurences[transactions[i][j]] += 1
        
mostPopular = sorted(occurences, key=occurences.__getitem__, reverse=True)

In [24]:
def getDataForTMostPopular(transactions, mostPopular, T):
    order = {mostPopular[i] : i for i in range(T)}
    
    p = np.zeros((T,T))
    
    for i in range(len(transactions)):
        fromTMostPop = [order[id] for id in transactions[i] if id in order]
        for j in fromTMostPop:
            for k in fromTMostPop:
                p[j][k] += 1 if j != k else 0
    
    return p, order

In [33]:
T = 1000
data, order = getDataForTMostPopular(transactions, mostPopular, T)
print("Data generated")

Data generated


In [32]:
groups, centers = KMeans(5, data)

Iterations: 30
K-Means done in 0.14215469360351562 seconds
